In [1]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
import scipy.misc as mi
from keras.models import model_from_json


Using TensorFlow backend.


In [2]:
def loadThatModel(folder):
    with open(folder+".json", 'rb') as f:
        json_string = f.read()
    model = model_from_json(json_string)
    model.load_weights(folder+".h5")
    return model

### 模型加载

In [3]:
# define some run parameters
maxToAdd = 8
hidden_units = 150
examplesPer = 10
size = 28

model = loadThatModel("../models/basicRNN")

FileNotFoundError: [Errno 2] No such file or directory: '../models/basicRNN.json'

In [5]:
# the data, shuffled and split between train and test sets
(X_train_raw, y_train_temp), (X_test_raw, y_test_temp) = mnist.load_data()

11493376/11490434 [==============================] - 270s 23us/step


In [ ]:
# basic image processing
X_train_raw = X_train_raw.astype('float32')
X_test_raw = X_test_raw.astype('float32')
X_train_raw /= 255
X_test_raw /= 255

In [ ]:
# run epochs of sampling data then training
y_test = []
X_test = np.zeros((examplesPer, maxToAdd, 1, size, size))
inds = []
for i in range(0, examplesPer):
    output = np.zeros((maxToAdd, 1, size, size))
    numToAdd = np.ceil(np.random.rand()*maxToAdd)
    indices = np.random.choice(X_test_raw.shape[0], size=numToAdd)
    example = X_test_raw[indices]
    exampleY = y_test_temp[indices]
    output[0:numToAdd, 0, :, :] = example
    X_test[i, :, :, :, :] = output
    y_test.append(np.sum(exampleY))
    inds.append(indices)

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

### 模型预测

In [ ]:
preds = model.predict(X_test)
for num, ans in enumerate(y_test):
    images = np.zeros((maxToAdd, 1, size, size))
    xtr = X_test_raw[inds[num]]
    images[0:xtr.shape[0], 0, :, :] = xtr
    for num2, image in enumerate(images):
        print(image.shape)
        image = image[0, :, :]
        mi.imsave("../images/image"+str(num2)+".jpg", image)
    print(ans, preds[num])
    for num3 in range(0, 5):
        output = np.zeros((maxToAdd, 1, size, size))
        example = X_test[num][0:num3+1]
        output[0:num3+1, :, :, :] = example
        output = np.reshape(output, (1, output.shape[0], output.shape[1], output.shape[2], output.shape[3]))
        tempPred = model.predict(output)
        print("with ", num3, " images: ", tempPred)
    stop = input("")